In [ ]:
# import pandas as pd
# import glob

# # Define the folder path
# computer_villa = 'C:/Users/RuohanLi/Villanova University/Complete-trip-coordinate - General'
# file_paths = glob.glob(computer_villa + '/Salt_Lake/delivery/Salt_Lake-Mar-2020/*.snappy.parquet')
# df_list = [pd.read_parquet(file, engine='pyarrow') for file in file_paths]
# # Load the first file
# combined_df = pd.concat(df_list, ignore_index=True)

In [1]:
import pandas as pd
import glob
from datetime import timedelta
import pygeohash as pgh
import geopandas as gpd
from shapely.geometry import Point

# Folder path
folder_path = 'C:/Users/RuohanLi/Villanova University/Complete-trip-coordinate - General'

# Months
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Only load needed columns
use_cols = [
    'linked_trip_id', 'trip_id', 'travel_mode', 'local_datetime_start', 'local_datetime_end',
    'network_distance', 'geohash7_orig', 'geohash7_dest'
]
# Travel modes of interest
modes = {'car', 'bus', 'rail', 'walk/bike'}  # use set for faster membership check

# List to store DataFrames
df_list = []

for month in months:
    files = glob.glob(f"{folder_path}/Salt_Lake/delivery/Salt_Lake-{month}-2020/*.snappy.parquet")
    for file in files:
        df = pd.read_parquet(file, engine='pyarrow', columns=use_cols)
        df['linked_trip_id'] = df['linked_trip_id'].astype(str)
        df['trip_id'] = df['trip_id'].astype(str)
        df['travel_mode'] = df['travel_mode'].astype(str).str.lower().str.strip()

        # 3. 时间列转为 datetime
        df['local_datetime_start'] = pd.to_datetime(df['local_datetime_start'], errors='coerce')
        df['local_datetime_end'] = pd.to_datetime(df['local_datetime_end'], errors='coerce')

        # 4. 过滤非法时间
        df = df[df['local_datetime_end'] > df['local_datetime_start']]

        # 5. 计算 duration
        df['duration_minutes'] = (df['local_datetime_end'] - df['local_datetime_start']).dt.total_seconds() / 60

        # 6. 清洗空间列（geohash）
        df['geohash7_orig'] = df['geohash7_orig'].astype(str).str.strip()
        df['geohash7_dest'] = df['geohash7_dest'].astype(str).str.strip()

        # 7. 清洗距离字段（避免非数字）
        df['network_distance'] = pd.to_numeric(df['network_distance'], errors='coerce')
        df = df[df['network_distance'] > 0]
        df_list.append(df)

all_df = pd.concat(df_list, ignore_index=True)



In [2]:
# ============ 2. 加载地理边界并筛选Geohash对应关系 ============
tract_path = r"C:\Users\RuohanLi\Villanova University\Complete-trip-coordinate - General\Manuscript\Figure\Visualization-RL\2-OD patterns by census track\six_counties_track.shp"
tracts = gpd.read_file(tract_path).to_crs("EPSG:4326")

# Step 1: 构造 orig 和 dest 点
gdf_orig = gpd.GeoDataFrame(
    all_df[['geohash7_orig']],  # 只用 orig 部分
    geometry=all_df['geohash7_orig'].apply(lambda gh: Point(pgh.decode(gh)[1], pgh.decode(gh)[0])),
    crs="EPSG:4326"
)
gdf_orig.index = all_df.index  # 保持 index 对齐

gdf_dest = gpd.GeoDataFrame(
    all_df[['geohash7_dest']],
    geometry=all_df['geohash7_dest'].apply(lambda gh: Point(pgh.decode(gh)[1], pgh.decode(gh)[0])),
    crs="EPSG:4326"
)
gdf_dest.index = all_df.index

# Step 2: 空间连接 tract shapefile
gdf_orig_joined = gpd.sjoin(gdf_orig, tracts[['GEOID', 'geometry']], how="left", predicate="within")
gdf_dest_joined = gpd.sjoin(gdf_dest, tracts[['GEOID', 'geometry']], how="left", predicate="within")

# Step 3: 把 GEOID 信息通过 index merge 回 all_df
all_df['GEOID_orig'] = gdf_orig_joined['GEOID']
all_df['GEOID_dest'] = gdf_dest_joined['GEOID']


In [7]:
import pandas as pd
import glob
import pygeohash as pgh
from shapely.geometry import LineString
import geopandas as gpd
from datetime import datetime

# 加载并清洗数据：略（使用你已有的 all_df）

# === 第一步：匹配 linked_trip 起点终点 ===
sorted_df = all_df.sort_values(['linked_trip_id', 'local_datetime_start'])

first_trips = sorted_df.groupby('linked_trip_id').first().reset_index()
last_trips = sorted_df.groupby('linked_trip_id').last().reset_index()

merged = first_trips[['linked_trip_id', 'GEOID_orig']] \
    .merge(last_trips[['linked_trip_id', 'GEOID_dest']], on='linked_trip_id')

target_linked = merged[
    (merged['GEOID_orig'] == '49035114000') &
    (merged['GEOID_dest'] == '49035110106')
]['linked_trip_id'].unique()

target_df = sorted_df[sorted_df['linked_trip_id'].isin(target_linked)]

# === 第二步：识别符合 multimodal 的 trip ===
def is_multimodal(modes):
    seq = list(modes)
    mode_set = set(seq)
    # walk-bus-walk
    # car + (bus or rail)
    if 'car' in mode_set and 'walk/bike' in mode_set :
        return True
    return False

filtered_trips = []
for ltid, group in target_df.groupby('linked_trip_id'):
    modes = list(group.sort_values('local_datetime_start')['travel_mode'])
    # if is_multimodal(modes):
    filtered_trips.append(ltid)

sample_df = target_df[target_df['linked_trip_id'].isin(filtered_trips)]

# === 第三步：生成结果表 ===
output = []
for ltid, group in sample_df.groupby('linked_trip_id'):
    group = group.sort_values('local_datetime_start')
    mode_seq = '->'.join(group['travel_mode'])

    # 经纬度坐标序列
    coords = []
    times = []
    for _, row in group.iterrows():
        try:
            o_lat, o_lng = pgh.decode(row['geohash7_orig'])
            d_lat, d_lng = pgh.decode(row['geohash7_dest'])
            coords.append((o_lng, o_lat))  # shapely: (x, y) = (lon, lat)
            coords.append((d_lng, d_lat))
            times.append(row['local_datetime_start'])
            times.append(row['local_datetime_end'])
        except:
            continue

    if len(coords) < 2:
        continue

    linestring = LineString(coords)
    times_str = '->'.join([t.strftime('%Y-%m-%d %H:%M:%S') for t in times])

    output.append({
        'linked_trip_id': ltid,
        'mode_sequence': mode_seq,
        'geometry_wkt': linestring.wkt,
        'time_sequence': times_str
    })

# === 第四步：保存结果 ===
output_df = pd.DataFrame(output)
output_df.to_csv('multimodal_trip_sample.csv', index=False)


# specific sample

In [15]:
import pandas as pd
import os
import glob
import pygeohash as pgh
from shapely.geometry import LineString
from shapely import wkt as shapely_wkt
import ast

# === 设置 ===
folder_path = "C:/Users/RuohanLi/Villanova University/Complete-trip-coordinate - General"
target_ids = [
    "KWzJGrJMlbBXGBdYWd51rRYNrlVBRNYJQLwGZylWa4lZeKq0oP0LQE-rVzWQ05jPj2X9LLBQXpAvdqLrg5QX",  # Jan
    "RLaKOp9YB2WDo44DoYLLKbeKjqzoWAz9J7vM7wEJxEDBwZExB262rO-1B1XElqEwldbAnxYQlwvwAqDx1xJg",  # Feb
]

# === 加载 Jan 和 Feb 的全部数据 ===
months = ['Jan', 'Feb']
use_cols = ['linked_trip_id', 'trip_id', 'geohash7_orig', 'geohash7_dest', 'route_taken', 'travel_mode']
df_list = []

for month in months:
    files = glob.glob(os.path.join(folder_path, f"Salt_Lake/delivery/Salt_Lake-{month}-2020/*.snappy.parquet"))
    for file in files:
        df = pd.read_parquet(file, engine='pyarrow', columns=use_cols)
        df = df[df['linked_trip_id'].isin(target_ids)]
        df_list.append(df)

target_df = pd.concat(df_list, ignore_index=True)

# === OD坐标列 ===
def decode_geohash_pair(row):
    try:
        lat1, lon1 = pgh.decode(row['geohash7_orig'])
        lat2, lon2 = pgh.decode(row['geohash7_dest'])
        return pd.Series([lon1, lat1, lon2, lat2])
    except:
        return pd.Series([None, None, None, None])

target_df[['orig_lon', 'orig_lat', 'dest_lon', 'dest_lat']] = target_df.apply(decode_geohash_pair, axis=1)

# === 加载路网并构建 mode-specific 字典 ===
link_car = pd.read_csv(os.path.join(folder_path, "Salt_Lake/supplementInputs/network/auto-biggest-connected-graph/link.csv"))
link_car_dict = {
    (int(row['from_osm_node_id']), int(row['to_osm_node_id'])): row['geometry']
    for _, row in link_car.iterrows()
}

link_transit = pd.read_csv(os.path.join(folder_path, "Salt_Lake/supplementInputs/network/UTA/link with flow.csv"))
link_transit_dict = {
    (int(row['from_node_id']), int(row['to_node_id'])): row['geometry']
    for _, row in link_transit.iterrows()
}

# === 构造 trip-level 汇总结果 ===
output_rows = []
for ltid, group in target_df.groupby('linked_trip_id'):
    all_geoms = []
    for _, row in group.iterrows():
        try:
            # 解析节点序列
            nodes = [int(n.strip()) for n in row['route_taken'].split(',') if n.strip().isdigit() and int(n.strip()) != -1]
            if len(nodes) < 2:
                continue

            # 根据 mode 选取 link_dict
            mode = row['travel_mode'].lower()
            if mode == 'car':
                link_dict = link_car_dict
            elif mode in ['bus', 'rail']:
                link_dict = link_transit_dict
            else:
                continue  # skip walk/bike/air etc.

            # 查找 link geometry
            for i in range(len(nodes)-1):
                pair = (nodes[i], nodes[i+1])
                if pair in link_dict:
                    try:
                        linestring = shapely_wkt.loads(link_dict[pair])
                        all_geoms.extend(list(linestring.coords))
                    except Exception as e:
                        print(f"❌ WKT parse error for pair {pair}: {e}")
        except Exception as e:
            print(f"❌ Error in trip {row['trip_id']}: {e}")
            continue

    if all_geoms:
        full_linestring = LineString(all_geoms)
        output_rows.append({
            'linked_trip_id': ltid,
            'trip_count': len(group),
            'full_geometry_wkt': full_linestring.wkt
        })

# === 合并结果并保存 ===
if output_rows:
    output_df = pd.DataFrame(output_rows)
    target_df = target_df.merge(output_df, on='linked_trip_id', how='left')
else:
    print("⚠️ No link geometries were matched. Adding empty column.")
    target_df['full_geometry_wkt'] = None

target_df.to_csv("selected_trips_with_geometry.csv", index=False)
print("✅ Done. File saved as selected_trips_with_geometry.csv.")


✅ Done. File saved as selected_trips_with_geometry.csv.


In [17]:
import pandas as pd
import glob
import os

# 设置路径和 ID
folder_path = "C:/Users/RuohanLi/Villanova University/Complete-trip-coordinate - General"
target_ids = [
    "KWzJGrJMlbBXGBdYWd51rRYNrlVBRNYJQLwGZylWa4lZeKq0oP0LQE-rVzWQ05jPj2X9LLBQXpAvdqLrg5QX",  # Jan
    "RLaKOp9YB2WDo44DoYLLKbeKjqzoWAz9J7vM7wEJxEDBwZExB262rO-1B1XElqEwldbAnxYQlwvwAqDx1xJg",  # Feb
]
months = ['Jan', 'Feb']

# 读取并筛选数据
df_list = []
for month in months:
    files = glob.glob(os.path.join(folder_path, f"Salt_Lake/delivery/Salt_Lake-{month}-2020/*.snappy.parquet"))
    for file in files:
        df = pd.read_parquet(file)
        filtered = df[df['linked_trip_id'].isin(target_ids)]
        if not filtered.empty:
            df_list.append(filtered)

# 合并并保存
if df_list:
    result_df = pd.concat(df_list, ignore_index=True)
    result_df.to_csv("selected_linked_trips_jan_feb.csv", index=False)
    print("✅ Saved to selected_linked_trips_jan_feb.csv")
else:
    print("⚠️ No matching linked_trip_id found in Jan or Feb.")


✅ Saved to selected_linked_trips_jan_feb.csv
